In [ ]:
! conda install bs4 lxml --yes


# WebScraping
## 1. Extracting one Row of Data

### Reviewing the source

- First we take a look at the page of HTML we want to capture. In a seperate window open up http://www.uberpeople.net/forums/Tips.

- Explore the page as it is rendered in the browser, and the underlying code by right clicking on the page and using 'View Page Source'.

- For now, we're just going to pull this entire page into memory and then we'll work out how to extract the parts we want.


To retrieve the HTML we're going to use the Requests library which we will import below.

The documentation for Requests can be found at http://docs.python-requests.org/en/master/


In [ ]:
# Due to website design we will need to present ourselves as a web browser
# Using the file of 'user-agent' data we can send the correct headers when making our requests 
from random import choice
with open('user_agent.txt','r') as f:
    agents = f.readlines()
    agents = [x.strip() for x in agents]

In [ ]:
import requests

In [ ]:
response = requests.get('http://uberpeople.net/forums/Tips/', headers={'user-agent': choice(agents)}) # Yes it is that simple - thanks Requests!

In [ ]:
#  If we look at our html object we get a simple response code. 200 is a success, 404 for example would be a failure.
#  For a full list of Http response codes see https://httpstatuses.com
response

In [ ]:
# We can look at the content of the retrieved package. 
# Click the bar to the left of the text to expand or contract its screen usage.
response.text

### Inspecting your source
- Ok that big block of mess isn't that helpful...
- We need to find a systematic way of combing through the entire HTML, and picking out what we need.
- Make sure the page is open in Chrome or Firefox and then right click on the first title and choose 'Inspect (Element)' to see the underlying code.

We can see that each row is in its own division. All these divisions sit inside a parent division with the class `"structItemContainer-group js-threadList"`. Knowing this will let us drill down into each row, and later iterate over each row and perform the same actions. To do this we will use a library called **Beautiful Soup**.

Documentation for Beautiful Soup https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [ ]:
# First we get Beautiful soup to break down the HTML into something that can be navigated.
from bs4 import BeautifulSoup 

soup = BeautifulSoup(response.text,'lxml') # we have to make sure we give it the html.text content, and define the parser.

In [ ]:
# If we look at our soup it is a little more structured, but lets keep refining....
print(soup.prettify())

In [ ]:
# Let's first focus in on the section of page we want - the division containing all the thread entries
threads_container = soup.find('div', {'class':"structItemContainer-group js-threadList"})
threads_container

In [ ]:


# if we take our thread_frame we can use 'find_all' to return a list of child elements that match our criteria.
# Each row has multiple classes, we will just pass the one that seems specific to rows of the table.

threads = threads_container.find_all('div',{'class':'structItem--thread','data-author':True})
threads

In [ ]:
# We can check this has worked and not given us any more than  the rows by counting the number of rows on the page 
# (20) and checking against the length (len) of the list here...
len(threads)

In [ ]:
# Now let's find that first list item i.e. the first row.
first_item = threads[0]

print(first_item.prettify())

### Extracting row items
The items we want from this row are...
- Author
- Thread-id > useful for ensuring no duplicates and for quickly locating threads later.
- Title
- Date
- URL

#### Author

In [ ]:
# Author is located inside the division acts as the container for our first item.
# it is known as an attribute of the division called data-author. 
# We can retrieve the content of a tag attrribute as if the tag were a dictionary.

author = first_item['data-author']
print(author)

#### Thread_id

In [ ]:
# Unique IDs are not necesarily present in all websites, but this site happens to use them
# It's not necessarily clear straight away from the code exactly what counts as the id.
# making these decisions often requires you to look around the site and
# get a feel for its structure.

# In this case we can see the same number being used in the top level row division, and in the 
# url for the thread content. We could extract this from either division but here we'll take it
# from the row data, where the id is in the 'class' 

first_item['class']

In [ ]:
# class has multiple elements which beautifulsoup returns as a list,
# we need the last item in the list
id_item = first_item['class'][-1]
id_item

In [ ]:
# the last item is a string and we just need everything after the '-'
# we split up the string on the '-'...
id_item.split('-')

In [ ]:
# grab the last item...
id_item.split('-')[-1]

In [ ]:
# and convert it into an integer rather than keep the string
thread_id = int(id_item.split('-')[-1])
thread_id

For the remainder of items we need to step into the sub-divisions of the element, the div's inside our div. The row is made up of multiple subsections containing the information we want so we will need to step from our top level division into the various subsections.

#### Title

In [ ]:
# starting with our first_item we use .find to search within its subordinates to find the
# division containing the title.

title_div = first_item.find('div', class_='structItem-title')
title_div

In [ ]:
# inside the title division is a url division which is always tagged with the <a> tag. 
# we can access this either with .find('a') or the convenience method of simply .a which
# selects the first child element that is an <a> tag.

title_div.a

In [ ]:
# .text allows us to just get the plain text <a> between the tags </a>
title = title_div.a.text
title

In [ ]:
#.strip() is a string function that cleans up the ends removing white space and breaks.
# it is a good precautinary measure when gathering text from sites to ensure you aren't collecting
# erroneous whitespace.

title = title.strip()

#### Date

In [ ]:
# if we inspect the date we can see it sits within a <time> tag within our row division
first_item.find('time')

In [ ]:
# Time is represented in a lot of different ways here, and Dates and times 
# are special types of object because they do not behave like normal numbers, 
# nor are they useful only as strings. The best thing to do is to save the string
# called 'datetime' and convert it from a string into a 'datetime' object.


date_string = first_item.find('time')['datetime']
print(date_string)
print(type(date_string))

In [ ]:
# Currently this is a string, which will be problematic later if we want any analysis to
# understand it as time.

# We can convert it from a string to a datetime object using the datetime library

from datetime import datetime

# we need to instruct the datetime .strptime function what parts of the string pertain to 
# the different divisions of time. For this site our instruction looks like this...

date_format = '%Y-%m-%dT%H:%M:%S%z'
# the string instructs strptime which parts of the string 
# are time components and which parts are just string such as dashes and colons.
# See https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior 
# for full documentation



# %Y - Year
# %m - Month
# %d - day
# %H - Hour
# %M - Minute
# %S - Second
# %z - Timezone offset from GMT in hours

date = datetime.strptime(date_string, date_format )
print(date)
print(type(date))

#### URL

In [ ]:
# to get to each thread, the user would click the title of the thread,
# meaning the url for the thread must be in the title division somewhere
title_div

In [ ]:
# yes it is in the href attribute of the child <a>
title_div.a['href']

In [ ]:
# But this is not a whole url, it's relative to the domain 'http://uberpeople.net'

In [ ]:
# So let's put it all together
relative_url = title_div.a['href']
url = 'http://uberpeople.net' + relative_url
url

In [ ]:
# However the safe way to do it, because URLS can go wonky sometimes is to use part of the standard library.
import urllib
url = urllib.parse.urljoin('http://uberpeople.net', relative_url)
# This has some verification features to make sure the URL makes sense.
print(url)

### Putting it all Together

In [ ]:
def row_info_extractor(row): # We'll feed it the isolated html for a row and let it pull it apart.
    
    #author
    author = row['data-author']
    
    #id
    id_item = row['class'][-1]
    thread_id = int(id_item.split('-')[-1])
    
    #title
    title_div = row.find('div', class_='structItem-title')
    title = title_div.a.text.strip() # remember to .strip() off the useless spaces on the ends.
    
    #date
    date_format = '%Y-%m-%dT%H:%M:%S%z'
    date_string = row.find('time')['datetime']


    date = datetime.strptime(date_string, date_format)
    
    #url
    relative_url = title_div.a['href']
    # remember the url is only relative so it needs to be made full using urlib.parse.urljoin
    full_url = urllib.parse.urljoin('http://uberpeople.net',relative_url)
    
    # And now we spit out our final product - in this case we'll go for a list for Pandas use later.
    # We'll also re-order
    data_package = {'author':author,
                   'title':title,
                   'thread_id':thread_id,
                   'date':date,
                   'url':full_url}
    
    return data_package

In [ ]:
# Let's try it out

soup = BeautifulSoup(response.text,'lxml')
threads_container = soup.find('div', class_="structItemContainer-group js-threadList")
threads = threads_container.find_all('div', {'class':'structItem--thread', 'data-author':True} )

first_item = threads[0]


row_info_extractor(first_item)

In [ ]:
data = []
for row in threads:
    row_info = row_info_extractor(row)
    data.append(row_info)

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df